# Plan de trabajo UdeA
## Installation
1. Donload this `notebook` by following [this link](https://raw.githubusercontent.com/restrepo/plan_de_trabajo/main/ptd.ipynb)

2. Save with: <kbd>Ctrl</kbd>+<kbd>S</kbd>

3. Remove:  `.txt` 

 [![enter image description here][1]][1]

4. Download and install Anaconda from [here](https://www.anaconda.com/download/success), and from the "Anaconda Navigation" window, choose "JupyterLab":
   
![image](https://github.com/restrepo/plan_de_trabajo/assets/655883/860c34b0-4052-43a7-82f9-0d577cc3c528)

5. Now navigate to the directory where `noebook` is downloaded in "JupyterLab"  and open it.

6. Use <kbd>Shift</kbd>+<kbd>Enter</kbd> to execute each cell, sequantially


  [1]: https://i.stack.imgur.com/Av30W.png

In [1]:
try:
    import helium
except:
    import os
    os.system('pip install helium') 
    os.system('conda install -c conda-forge ipywidgets') #jupyterlab_widgets also intalled
    print("Restart of the notebook is requiered. From the menu: `File → Shut Down` and open again from 'Anaconda Navigator'" )

In [2]:
# Get python file from GitHub if not avalaible
from os.path import isfile
py_file = 'plan_de_trabajo.py'
if not isfile(py_file):
    import requests
    
    fpy = requests.get(f'https://raw.githubusercontent.com/restrepo/plan_de_trabajo/main/{py_file}')
    
    if fpy.status_code == 200:
        f = open(py_file,'w',encoding="utf8")
        f.write(fpy.text)
        f.close()

In [1]:
from plan_de_trabajo import *
# For not interactive mode, define `defaults` list:
#  `defaults` must be defined according to the order of radios and drops keys below, and can be used outside jupyter
radios,drops = get_config(defaults=[True, True, False, False, False,'Todos','Aprobado','2022-1']) # Usually (defaults = [])
file = 'ptd_fcen_20221.json' # '' #

RadioButtons(description='Salvar la base de datos', options=(True, False), value=True)

RadioButtons(description='Actualizar la base de datos', options=(True, False), value=True)

RadioButtons(description='Enviar el plan de trabajo al profesor', index=1, options=(True, False), value=False)

RadioButtons(description='Autenticarse de nuevo', index=1, options=(True, False), value=False)

RadioButtons(description='RESET i: ignorando la base de datos', index=1, options=(True, False), value=False)

────────────────────────────────────────────────────────────────────────────────


Dropdown(description='Profesores', index=1, options=('Instituto', 'Todos'), value='Todos')

Dropdown(description='Estado', index=1, options=('Autorizado', 'Aprobado', 'Diligenciado', 'Devuelto para revi…

Dropdown(description='Semestre', index=4, options=('2024-1', '2023-2', '2023-1', '2022-2', '2022-1', '2021-2',…

In [ ]:
SAVE_DATABASE = radios['Salvar la base de datos'].get_interact_value()
UPDATE_DATABASE = radios['Actualizar la base de datos'].get_interact_value()
ENVIAR = radios['Enviar el plan de trabajo al profesor'].get_interact_value()
TODOS = False
if drops['Grupo de profesores'].get_interact_value() == 'Todos':
    TODOS = True
NEW_LOGIN = radios['Autenticarse de nuevo'].get_interact_value()
RESET_i = radios['RESET i: ignorando la base de datos']
ESTADO = drops['Estado del plan de trabajo'].get_interact_value()
SEMESTRE = drops['Semestre'].get_interact_value()
if not file:
    file = input('Nombre para el archivo de salida en json. e.g:\nptd_fcen_20222.json\n')

if not NEW_LOGIN:
    driver = hell.start_chrome()
#else:
#    raise Exception('set NEW_LOGIN = False')
    
login()

print('wait for 5 sec')
sleep(5)
lptd = []
if UPDATE_DATABASE:
    try:
        f=open(file,'r')
        lptd = json.load(f)
        f.close()
    except:
        pass
print(f'{len(lptd)} records in database' )


guess_page = int(len(lptd)/20)+1


#TODO: Calculate i_page fron len(lptd))
print(f'Incialización para {file} con len: {len(lptd)} e i_page = {guess_page}')
ptd = PTD(i_page=guess_page)    # Clean object

#while True: # ptd.get_docente() genera error cuando no hya más autores
ptd.estado = ESTADO
ptd.semestre = SEMESTRE
ptd.todos = TODOS # Comment for False
#if ptd.todos and ENVIAR: #TODO → Check for Decano submissions
#    ENVIAR = False
# TODO: (Semestre*,YYYY-S),  (Centro gestor, Todos) 
if ptd.NEXT_STEP:
    ptd.gestionar_planes(institutos = ['BIOLOGIA','FISICA','MATEMATICA','QUIMICA', 'CIENCIAS DEL MAR'],SINGLE = False)
else:
    raise Exception('Búsqueda fallida')

hell.wait_until(hell.Text('Fecha inicio semestre').exists,timeout_secs=180)
ptd.go_to_initial_page() #TODO: Define self.i_page here from guess_page

# TODO: i cannot depend on lptd if lptd is updated in a new run
RESET_i = False # Configuratins at beggining
if RESET_i:
    i=0
    ptd.i_page = 0
else:
    i = len(lptd)-(ptd.i_page-1)*ptd.n_page # For new data append to old database

# Obtain registros totales (Default: ptd.n_total = 1000)
try:
    ptd.n_total = eval( hell.get_driver().find_element(
        By.CLASS_NAME , 'col-md-2').text.split('\n')[0] )
except:
    pass

i_max , i_page_max = ptd.get_max_indices()
print(f'{ptd.n_total}: {i_max} : {i_page_max}')


for j in range(5):
    print('va',i)
    if hell.Text('Volver').exists():
        hell.click('Volver')
    try:
        print('in loop:',i)
        i,lptd,ptd = loop(i, lptd, ptd, SAVE_DATABASE, ENVIAR, file)
    except:
        if i == i_max and ptd.i_page == i_page_max:
            raise Exception('The final End')        
        print('j',j)
        print('wait 60 seconds...')
        sleep(60)
        if not hell.Text("Gestión de planes de trabajo").exists():
                if hell.Text('CONTINUAR').exists():
                    hell.click('CONTINUAR')
                if hell.Text('Aceptar').exists():
                    hell.click('Aceptar')
                if hell.Text('Volver').exists():
                    hell.click('Volver')
                print('va')
    
        # Login again
        if not hell.Text("Gestión de planes de trabajo").exists():
            #raise Exception('Check what is happening')
            #check_login_page = input('Check if login is necessary (Y/N):')
            #if check_login_page == 'Y' or check_login_page == 'y' or check_login_page == 'yes':
            if not hell.Text("Gestión de planes de trabajo").exists() and not hell.Text('Fecha inicio semestre').exists():
                #Probably log out
                login()
                url="https://ayudame2.udea.edu.co/php_app/?app=inicio&appid=PLANDOCEN"
                hell.go_to(url)
                # Uses current configuration
                ptd.gestionar_planes(institutos = ['BIOLOGIA','FISICA','MATEMATICA','QUIMICA', 'CIENCIAS DEL MAR'],SINGLE = False)
                hell.wait_until(hell.Text('Fecha inicio semestre').exists,timeout_secs=180)
                ptd.go_to_initial_page()
        if not RESET_i:
            print(f'previous i {i}')
            i = len(lptd)-(ptd.i_page-1)*ptd.n_page + 1 # set i inside except:
            print(f'new i {i}')
        
    if i < 0:
        raise Exception('ERROR: Negative i, restart kernel and run again')
    #TODO: How to break
    if i == i_max and ptd.i_page == i_page_max:
        raise Exception('The End')

Using existing local encrypted login file from previos run: ./login.enc


Captcha solved? (Hit <Enter> 


wait for 5 sec
78 records in database
Incialización para ptd_fcen_20221.json con len: 78 e i_page = 4
180: 19 : 9nds... 4                                                             
va 18
in loop: 18
page: 4; i: 18                                                                  
Docente: ISABEL
Already in database
page: 4; i: 19                                                                  
Docente: ALEXANDER
Already in database
page: 4; i: 20                                                                  
Docente: LUCY                                                                   
Already in database
page: 5; i: 1                                                                   
Docente: ALEJANDRO
page: 5; i: 2 ...                                                               
Docente: ANDRES
page: 5; i: 3 ...                                                               
Docente: ALBERTO
page: 5; i: 4 ...                                                               
Doce

In [ ]:
! git commit -am 'lc'
! git push

__________________

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

TODO: Use [`textualize`](https://textual.textualize.io/) when not in Jupyter:
```python
# How can I check if code is executed in the IPython notebook?
#https://stackoverflow.com/a/39662359
get_ipython().__class__.__name__
```

## Activities
Manually check activities and creates a dictionary with the observations to be added

TODO:

```python
observaciones = [
{'identificación_1':'texto de observaciones 1'},
{'identificación_2':'texto de observaciones 2'},
...
]
```

In [9]:
ptd.get_actividades()

In [11]:
p{usuario}td.actividades.keys()

dict_keys(['actividades_relacionadas_con_la_docencia', 'actividades_relacionadas_con_la_investigación', 'actividades_relacionadas_con_la_extensión', 'otras_Actividades'])

In [16]:
ptd.actividades['actividades_relacionadas_con_la_investigación']

,Centro gestor,Actividad,Descripción,Semestre,Horas planeadas,identificación
0,instituto de fisica,Preparación de productos derivados de investigación,Atención visita Prof. José Mejía López de la Pontificia Católica de Chile y Preparación productos de investigación,2024-1,50,71760928
1,instituto de fisica,Reuniones de grupos de investigación,Reuniones grupos de investigación,2024-1,10,71760928
2,instituto de fisica,Coordinación grupo y/o proyecto de investigación,Coordinación del grupo de Física y Astrofísica\nComputacional FACOM,2024-1,48,71386935
3,instituto de fisica,Preparación de proyectos de investigación,"Investigador en proyecto ""MODELOS DE LÍNEAS DE EMISIÓN EN DISCOS PROTOPLANETARIOS PARA OBSERVACIONES CON EL JWST"", aprobado como elegible en convocatoria programática CODI FCEN 2021-22, actualment...",2024-1,100,80073361


In [17]:
ptd.actividades['otras_Actividades']

,Centro gestor,Actividad,Descripción,Semestre,Horas planeadas,identificación
0,instituto de fisica,Dedicación exclusiva,Dedicación exclusiva,2024-1,100,71760928
1,instituto de fisica,Claustro de profesores,Asambleas y claustros,2024-1,10,71760928
2,instituto de fisica,"Reuniones de departamento, sección, administración o unidad académica",Reuniones disponibilidad administrativa,2024-1,90,71760928
3,instituto de fisica,Dedicación exclusiva,"Dedicacion exclusiva para el proyecto ""Probando modelos de gravedad modificada con los datos de GAIA DR3""",2024-1,90,71386935
4,instituto de fisica,Evaluación de material académico-investigativo,"Permanentemente se me asigna como evaluador de propuestas de tesis de pregrado de astronomía, tesis de pregrado del instituto de física para distinciones, propouestas de investigación y tesis de p...",2024-1,27,71386935
5,instituto de fisica,Actividades de apoyo a la gestión académica-administrativa,Apoyo en el comité de posgrado del Instituto de Física\nAsistencia a reuniones de instituto\nAsistencia a reuniones de profesores de pregrado en astronomía\nAsistencia a reuniones de facultad\nOtr...,2024-1,45,71386935
6,instituto de fisica,Actividades de currículo y/o de rediseño curricular,Actividades Curriculares: Propuesta de v4 para pensum de Astronomía.,2024-1,76,80073361
7,instituto de fisica,Actividades de autoevaluación y acreditación,"Miembro de la comisión de acreditación de pregrados del Instituto, coordinador de autoevaluación y acreditación del pregrado de Astronomía",2024-1,150,80073361
8,instituto de fisica,Actividades de apoyo a la gestión académica-administrativa,"Apoyo en coordinación de programación de cursos del pregrado de Astronomía, participación en comité de pregrado por parte del pregrado de Astronomía.",2024-1,61,80073361
9,instituto de fisica,Representación o participación en Consejos o comités institucionales,Comité de género de la Facultad,2024-1,10,15370261


# Apéndice
## Lineamientos Plan de Trabajo
* [Lineamientos Plan de Trabajo.pdf](https://drive.google.com/file/d/1Podcl-IFFWmBEz2XZezyDhRIvKxykKXF/view?usp=sharing) (Aprobados en el Consejo de Facultad Acta 128 del 09 de septiembre de 2020)


1. Todo profesor deberá participar en la responsabilidad del desarrollo de un curso o de un componente curricular (art. 20 Estatuto Profesoral – AS 083/96).
2. → Al menos una de las siguientes actividades: __investigación__, producción académica (actividades relacionadas con la de investigación), capacitación (actividades relacionadas con la de docencia (ARLD)), actualización (ARLD) o __extensión__
3. __Actividades de Docencia__:
    1. Cursos dirigidos (código 96): __1 hora/semana__ por estudiante.
    2. Cursos regulares __hasta 25__ estudiantes: Por cada hora lectiva se asigna __1.75 horas de preparación__. por cada hora lectiva se asigna __0,5 horas por atención__.
    3. Cursos con entre __26 y 40__ estudiantes. Por cada hora lectiva se asignan __1.25  horas de preparación__, __además de 0.5 por atención__.
    4. Cursos semi-masivos: Es aquel que tiene entre __41 y 59__ estudiantes. Por cada  hora lectiva se asignan __1.5 horas de preparación__, además de __0.75 por  atención__.
    5. Cursos masivos: Es aquel que cuente con __60 o más__ estudiantes matriculados. Por cada hora lectiva se asigna __1.75 horas de preparación__, además de __1 por  atención__.
    6. Preparación de un curso nuevo: __Hasta 72 horas__ por curso teórico o teórico práctico
    7. Preparación de cursos que no se han dictado en los últimos 5 años: __18  horas adicionales__ al semestre por curso de núcleo medio y __36 horas  adicionales por curso avanzado o de posgrado__.
    8. Asesoría (o tutoría) de estudiantes de pregrado en semestre _previo a  matricular el Seminario de trabajo de grado_: __Hasta 18 horas__ al semestre  por estudiante y un máximo de tres estudiantes. Se debe entregar soporte de las reuniones y un  informe con las actividades desarrolladas por cada estudiante o el  proyecto de grado cuando corresponda.
    9. Producción de _material de docencia_: __Hasta 90 horas__ al semestre. El  docente debe presentar el entregable concertado con el Director, para ser  enviado a evaluación.
    10. Docente Enlace: Se refiere a aquellos profesores que, junto con un asesor externo, orientan trabajos de grado en cualquier modalidad. Se reconocen __hasta 10 horas__ al semestre por estudiante y un máximo de tres  estudiantes.
    11. Comités tutoriales de posgrado (diferentes a la Dirección de Trabajos de  grado): __Hasta 10 horas__ al semestre por estudiante. El comité debe redactar  y presentar un informe semestral dando cuenta del avance del estudiante.  Un profesor puede pertenecer a un máximo de tres comités.
    12. Trabajos de grado en otras dependencias: La dependencia debe cargar el  curso al docente con su respectivo número de horas. Se reconocen las  horas de acuerdo al registro de MARES. No se reconocen horas cuando  el estudiante no haya matriculado el curso.
    13. Trabajos de grado con Universidades con las cuales se tiene convenio: el  estudiante o el profesor debe legalizar la movilidad en la oficina de  Relaciones Internacionales, para que el curso quede registrado en MARES con sus respectivas horas. No se reconocen horas por lo que no  quede allí registrado.
    14. MARES con sus respectivas horas. No se reconocen horas por lo que no  quede allí registrado. Trabajos de grado declarados en Incompleto: Hasta 18 horas al semestre  por una única vez. Asesoría de estudiantes de posgrado en prórroga: __Hasta 18 horas__ al  semestre.
    15. Reuniones de grupo: __Hasta 20 horas__ por semestre
4. __Actividades de Investigación__:
    5. Proyectos aprobados por CODI, a través de convenios con entidades  externas o registrados y aprobados en el SIIU: Se reportan las  horas/semana que hayan sido avaladas cuando se aprobó el proyecto. El  plan de trabajo corresponde a __40 horas__ semana por 22.5 semanas para  un total de 900 horas semestre. Sin embargo, nótese que la plataforma  automáticamente multiplica por 26 semanas. Por lo tanto, se debe hacer  la corrección.
    6. Las horas avaladas incluyen la elaboración de manuscritos e informes del  proyecto. Un docente tendrá un máximo de 30 horas semana  (equivalentemente, 675 horas al semestre) dedicadas a la investigación.  Así que debe enseñar un al menos un curso y dejar __45 horas__ de  disponibilidad administrativa.
    7. En el Plan de Trabajo se debe consignar el nombre del proyecto, acta de  inicio, fecha de inicio y rol del profesor en el proyecto (investigador  principal co-investigador).
    8. Para proyectos financiados por entidades externas, debe incluirse el  mismo número de horas que fueron avaladas y que constituyen la  contrapartida de la Universidad. 
5. __Actividades relacionadas con investigación__:
    5. Coordinación de grupo de Investigación: __Hasta 70 horas__
    6. Elaboración de proyectos de Investigación: __Hasta 60 horas__ al semestre  por la preparación de un proyecto y __hasta 90 horas__ al semestre por dos o  más proyectos
    7. Proyectos en prórroga: __Hasta el número de horas avaladas para el  proyecto__. Se debe presentar el aval de la prórroga.
    8. Participación como jurado de tesis de posgrado: __Hasta 16 horas__ al  semestre. Solo se reconocen horas por evaluación de tesis cuando la labor  haya sido asignada; se debe presentar la solicitud que le fue enviada al  profesor.
    9. Evaluación como par evaluador de manuscritos sometidos a revista  indexada: __Hasta 10 horas__ por artículo evaluado. Solo se reconocen las horas cuando se presente la solicitud enviada al profesor por parte del  editor de la revista.
    10. Escritura de artículos, libros y capítulos de libros que no estén adscritos a proyectos de investigación y sus entregables deben concertarse con el  director.

6. __Actividades de Extensión__:
    7. Proyectos inscritos en la plataforma de servicios de extensión, se reportan  las horas/semana que hayan sido avaladas cuando se aprobó el proyecto.  Para el total de horas semestre se multiplica por 22,5 semanas.
7. __Actividades relacionadas con Extensión__
    1. Para las actividades no inscritas en la plataforma de servicios de extensión  y formalizadas con contratos o convenios, se reportan las horas/semana  que hayan sido avaladas cuando se aprobó el proyecto. Por favor verifique  que el proyecto se encuentra vigente para el semestre en curso. Si culmina  antes de terminar el semestre, calcule las horas proporcionales al número  de semanas hasta la vigencia del proyecto.
    2. Coordinación de proyectos de Extensión: Se coloca el número de horas  avalado por el Consejo de Facultad multiplicado por 22.5.
    3. Coordinación de servicios de Extensión: Depende de la cantidad de  servicios que se presta y las horas se deben concertar con el director del  instituto. El profesor debe enviar un informe de los servicios prestados  durante el semestre.
    4. Preparación proyectos de Extensión: __Hasta 60 horas__ al semestre por la  preparación de un proyecto y __hasta 90 horas__ al semestre por dos o más  proyectos. El entregable consiste en el certificado de presentación del  proyecto a la convocatoria o al Centro de Extensión para inscripción en "Portafolio". Esto no aplica a los docentes con proyectos vigentes que  tengan como compromiso la presentación de un nuevo proyecto, en cuyo  caso las horas se incluyen dentro de la dedicación al proyecto.
    6. Asesoría y/o evaluación de proyectos de Extensión: Hasta 10 horas al  semestre. Solo se reconocen horas por evaluación de proyectos cuando  la labor haya sido asignada presentando la solicitud que le fue enviada al  profesor.
  
8. __ADMINISTRACIÓN ACADÉMICA__: 
    1. Coordinación de cursos de pregrado y posgrado: __hasta 20 horas__ por curso  y 4 horas adicionales por cada grupo. El número máximo por coordinación  es de 180 horas al semestre.
    2. Coordinación de programas de posgrado: __Hasta 225 horas__ al semestre  (aparece en los lineamientos de la dirección de posgrado).
    3. Coordinación de Extensión: __hasta 45 horas__ al semestre.
    4. Coordinación de Internacionalización: __hasta 45__ horas al semestre.
    5. Coordinación de prácticas: __hasta 150 horas__ al semestre para el  coordinador por instituto y hasta 300 para el coordinador de la facultad  (Acuerdo de facultad 172 de 2017).
    6. Coordinación de áreas: __hasta 45 horas__ al semestre (Acuerdo de Facultad  191 del 16 de marzo de 2017)
    7. Coordinación de Comités: según esté estipulado en el acto de creación del  Comité; de lo contrario se debe concertar con el Director de Instituto.
    8. Coordinación de regionalización: __hasta 300 horas__ al semestre.
    9. Dirección de revista: Se rige por RR 22863 del 9 de agosto de 2006.

9. __OTRAS ACTIVIDADES__
    1. Actividades de apoyo a la gestión académico administrativa: __45 horas__ al  semestre que incluyen reuniones de facultad, de instituto y reuniones  ordinarias de las áreas. Es de obligatoria inclusión.
    2. Actividades de currículo o diseño curricular: __hasta 45 horas__ semestre que  incluyen las actividades de autoevaluación, acreditación y revisión de  cursos. Sólo aplica a los profesores con actividades curriculares
    3. Docente enlace biblioteca. __Hasta 22,5 horas__ al semestre.
    4. Asistencia en procesos de admisión de pregrado: __hasta 12 horas__ al  semestre.
    5. Año Sabático: sólo lo reportan aquellos que tienen resolución para tomarlo.
    6. Comisión de estudio: sólo se registran una vez emitida la resolución  respectiva.
    7. Comisión administrativa externa sólo se registran una vez emitida la  resolución respectiva
    8. Representación institucional en consejos y comités sólo se registran una  vez emitida la resolución respectiva.
    9. Claustro de profesores: hasta 10 horas al semestre. 

8. __Capacitación__: Incluir las horas que se dedican a capacitación, especificando el nombre de  la actividad. Se requiere presentar cumplido al final del semestre. Se pueden  incluir hasta un 25% de horas adicionales a las presenciales, cuando el  programa lo amerite.  



### Definiciones
* *Actividades lectivas*: son modalidades de docencia que reúnen las siguientes características: son formativas, programadas, regulares, obligatorias para los estudiantes, evaluables, y su realización exige una preparación por parte del profesor. Comprenden la exposición y análisis en cursos, seminarios y talleres; la dirección o coordinación de talleres, seminarios, trabajos de campo o actividades prácticas profesionales; y la asesoría de monografías, tesis y trabajos de investigación y de grado. La disponibilidad para atender a los estudiantes no hará parte de la actividad lectiva
* *Cursos masivos*: se considera un curso masivo aquel que tenga matriculados más de 60 estudiantes.
* *Cursos semi-masivos*: Es aquel que tiene entre 41 y 59 estudiantes. 
* *Duración semestre*: En todos los casos se multiplica por 18, que es el número de semanas planeadas para el semestre.

pip install jupyterlab_myst

```{csv-table} Frozen Delights!
:header: "Treat", "Quantity", "Description"

"Albatross", 2.99, "On a stick!"
"Crunchy Frog", 1.49, "If we took the bones out
it wouldn't be crunchy, now would it?"
"Gannet Ripple", 1.99, "On a stick!"
```

### Heading Level 3

Try changing the number of `#`s to $n$[^math] to change the **depth** of the _heading_.

1. Learn about [Markdown](https://en.wikipedia.org/wiki/Markdown)
   - Go through a [tutorial](https://commonmark.org/help/tutorial/)

[^math]: Where $n \in \mathbb{N}$ with $n \leq 6$, or between an H1 and an H6!

```{figure} https://source.unsplash.com/random/500x200/?mountain
:label: my-fig
:align: center

My **bold** mountain 🏔🚠.
```

Check out [](#my-fig)!!